In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.9 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-fggl2emt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-fggl2emt
  Resolved https://github.com/huggingface/accelerate to commit eba6eb79dc2ab652cd8b44b37165a4852768a8ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=226907 sha256=7740646fdf4a87fa81d54fa43e107c849fb35bb11b9f89651e15ef8d9f12681c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ym_ou51r/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
from os import listdir

import pandas as pd
import numpy as np
import random
import re

import torch
from torch.utils.data import Dataset, random_split
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM, GPT2TokenizerFast
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
LYRICS_GENRE = 'Pop'

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP Finetuning/Genre Datasets/en_"+LYRICS_GENRE+"_Lyrics.csv")

print(data.shape)
data.head()

(33460, 3)


,Unnamed: 0,Lyric,Genres
0,0,I feel so unsure\nAs I take your hand and lead...,Pop
1,3,"Don't let them fool, ya\nOr even try to school...",Pop
2,6,"Baby, let's cruise, away from here\nDon't be c...",Pop
3,9,"Know it sounds funny\nBut, I just can't stand ...",Pop
4,12,You've got that look again\nThe one I hoped I ...,Pop


In [ ]:
data['length'] = data['Lyric'].apply(lambda x:len(str(x).split()))
data = data[data['length'] <= 200]

data = data.drop(['Unnamed: 0', 'Genres', 'length'], axis=1)

In [ ]:
data = data.head(2000)

In [ ]:
torch.cuda.empty_cache()

tokenizer = GPT2TokenizerFast.from_pretrained('EleutherAI/gpt-neo-125m', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  device = torch.device('cpu')

print(device)

cuda


In [ ]:
!nvidia-smi

Sun Jun  4 14:00:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train, test = train_test_split(data['Lyric'], test_size=0.15, shuffle=True, random_state=SEED)

In [ ]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = []
        self.input_ids = []
        self.attention_masks = []
        self.max_length = max([len(re.findall('\w+', x)) for x in data['Lyric']]) # Lyric with most words.
        for text in texts:
            dici = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
            self.input_ids.append(dici['input_ids'][0])
            self.attention_masks.append(dici['attention_mask'][0])

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [ ]:
train = ShapingDataset(train)
test = ShapingDataset(test)

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/outputs/',
    logging_dir = '/content/logs/',
    num_train_epochs = 20,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    logging_steps = 10000,
    save_steps = 10000,
    learning_rate = float('2e-4'),
    warmup_steps = 20)

In [ ]:
model.to(device)

trainer = Trainer(model=model, args=training_args,  train_dataset=train, eval_dataset=test, data_collator = lambda data: {
    'input_ids': torch.stack([f[0] for f in data]),
    'attention_mask': torch.stack([f[1] for f in data]),
    'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10000,1.192000
20000,0.120000
30000,0.028100


TrainOutput(global_step=34000, training_loss=0.39591205148135916, metrics={'train_runtime': 4247.4901, 'train_samples_per_second': 8.005, 'train_steps_per_second': 8.005, 'total_flos': 4544595357696000.0, 'train_loss': 0.39591205148135916, 'epoch': 20.0})

In [ ]:
saved_root = "/content/drive/MyDrive/NLP Finetuning/Finetuned Models/"+LYRICS_GENRE+" Lyrics Generator"

In [ ]:
trainer.save_model(saved_root)

In [ ]:
model = GPTNeoForCausalLM.from_pretrained(saved_root)

In [ ]:
model.to('cuda')
generated = tokenizer(' ', return_tensors = 'pt').input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=10, max_length=100, top_p=0.95, temperature=2., num_return_sequences=5)

for i, sample_output in enumerate(sample_outputs):
  text = tokenizer.decode(sample_output.tolist())
  regex_padding = re.sub('<|pad|>', '', text)
  regex_barra = re.sub('[|+]', '', regex_padding)
  space = re.sub('[ +]', ' ', regex_barra)
  result = re.sub('[\n](2, )', '\n', space)
  print("\n==== Generated lyric {}:\n\n{}".format(i+1, result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



==== Generated lyric 1:

 
I watch you sleeping
My weary heart rises up on wings
I hear you calling
But my heart has not been broken
You've defeated me yet
You've defeated me but I have never felt so right

Chorus:
So just take me for a ride
Let me take a ride
Till my broken heart sings
Take a ride, take a ride
And ride, ride, ride, take me
Show my broken heart howling
Through the night

==== Generated lyric 2:

 
Get up 
Don't again back down
irtyly

Playin' on the floor
Are you listening
To what I'm playing
Playin' on the floor
Listen to me now

Now the rockas down
Are you listening
To what I'm doing
Now the rockas down
Are you listening
To what I'm looking for

B-a-a-by
Baby take a seat
Let's rockas down
Are

==== Generated lyric 3:

 
Get up 
Don't again back on your feet 
To get up
Don't let nobody break you down 

Oh don't get the passion, control of your mind
Oh don't get frustration, announce of your time
And feel ebony, 
Your hopes to the crown
And baby don't stop your cry 
A